# Procure to Pay data transformation lab
In this example, we are going to transform a few data files provided by a client, into a multi-level process event-log that can be loaded and analyzed in IBM Process Mining.

This data transformation process can be done using various techniques and tools, and it is really a strength that clients can apply their existing data practices and skills to feed IBM Process Mining. 

The exact same transformation process is also illustrated in another lab using IBM SPSS. We could be using IBM Datastage, Talend, and so forth.

The goal of this example is to show how easy it is to do this work using Python language and Pandas library. Obviously, basic programming skills are required, but you will see that handling tables with Pandas (called dataframes) is quite simple, and that you can reuse parts of this example in your own project.

## Installing and running Jupyterlab
You need to have Python installed to run Jupyterlab locally. 
These steps should be sufficient to install Python and pandas on a Linux host:

`yum install -y python36-devel.x86_64`

`pip3 install jupyterlab`

`pip3 install pandas`

Then run:

`jupyter lab --allow-root`

Similar instructions exist for installing on Mac or Windows.

## What is Pandas
Pandas is an open source Python package that is most widely used for data science/data analysis and machine learning tasks. It is built on top of another package named Numpy, which provides support for multi-dimensional arrays. As one of the most popular data wrangling packages, Pandas works well with many other data science modules inside the Python ecosystem, and is typically included in every Python distribution.

## Problem Description
In this example, the client provided 3 CSV files that we want to transform to create a multi-level process event log for Procure To Pay (similar to the P2P event log used in the Process Mining Tutorials).

* requisition.csv contains the purchase requisition data set that needs to be transformed into an event log.

* procurement.csv contains ready-to-use events

* invoice.csv contains additional business data for invoices, that we want to add to the procurement events.


## requisition.csv
The requisition.csv file is not ready to be used by Process Mining. Indeed, it does not contain an activity column where activities are listed, nor does it contain a proper startDate column. Let's see what it looks like by loading it with pandas.

In [ ]:
import pandas as pd

requisitions = pd.read_csv('requisition.csv')
requisitions

The file includes 241133 requisitions, each with a Req_ID that indentifies each case.

There are two column named respectively 'Create_Date' and 'Release_DateTime' that represent the 'requisition creation' activities and dates, and 'requisition release' activity and date. They are associated with users, roles, type, and source specific to each activity.

In [ ]:
requisitions.columns

We are going to create the events for `Requisition Created` activity, and for `Requisition Released' activity`. We just need to duplicate the original dataset such that we have one event for each activity (remember, there are 2 columns that represent the activity dates), to add an 'activity' column and to rename a few columns.

### Requistion Created Events
Let's first manage the '`Requisition Created` events.

With Pandas, we copy the requisition dataframe, then we add an `activity` column. We fill this column with the string `Requisition Created`.

The field `Create_Date` becomes the `datetime` column that we keep for the entire P2P event log, as a start date. `Create_User` and `Create_Role` become respectively `user` and `role`.

We don't need to keep the fields and values related to the 'Release' activity in this dataset.

With Pandas, we use `dataframe.rename()` and `dataframe.drop()` functions. 

Alternatively, we could have created a new column `datetime`, and we would have filled it with the `Create_Date` column like this; `create_requisitions['activity'] = create_requisitions['Create_Date']`

In [ ]:
create_requisitions = requisitions.copy()
create_requisitions['activity'] = 'Requisition Created'
create_requisitions.rename(columns={'Create_Date' :'datetime', 'Create_User' : 'user', 'Create_Role' : 'role'}, inplace=True)
create_requisitions.drop(['Release_DateTime','Release_User','Release_Role','Release_Type','Release_Source'], axis=1, inplace=True)
create_requisitions.columns


### Requisition Released Events
We basically do the same thing with the `Requisition Released` fields

In [ ]:
release_requisitions = requisitions.copy()
release_requisitions['activity'] = 'Requisition Released'
release_requisitions.rename(columns={'Release_DateTime' :'datetime', 'Release_User':'user', 'Release_Role':'role'}, inplace=True)
release_requisitions.drop(['Create_Date','Create_User','Create_Role','Create_Type','Create_Source'], axis=1, inplace=True)
release_requisitions.columns

In [ ]:
# Do we have rows with nothing in the 'Create_Date' field? No. You can try the instruction below
# if we had, we could remove these rows with requisition[requisition['Create_Date'].notna()]
requisitions['Create_Date'].isna().sum()

### Append the 2 requisition events
With Pandas, we can easily append dataframes using `dataframe.append()`. The function does a union of the columns, and merging common columns, which is exactly what we need here.

In [ ]:
requisition_events = create_requisitions.copy()
requisition_events = requisition_events.append(release_requisitions)
requisition_events.columns

### Cleansing and saving the requisition events
In this dataset, each Req_ID has indeed the two activities. But it could happen that a Req_ID would not have the release information if this is a running case that has not yet reached this phase.

In this case, we would need to remove all the rows with a null value in the `datetime`column. That is done with the following code : `requisition_events[requisition_events['datetime'].notna()]`

`requisition_events['datetime'].notna()` returns False if the date is null. It acts as a filter with which we remove all the rows with a null date.

Let's see if there are some null dates, the function below should return 0 if no such rows.

In [ ]:
requisition_events['datetime'].isna().sum()

Finally we save the event log into a CSV file, and we keep the dataframe to append it later with the procurement dataset. The parameter ìndex=None' is needed to remove the first column (the index) that contains indexes for each row, which is not needed for process mining.

In [ ]:
requisition_events[requisition_events['datetime'].notna()] # just in case there are null dates
requisition_events.convert_dtypes() # Convert columns to best possible dtypes using dtypes supporting
requisition_events.to_csv('requisition_events.csv', index=None)
requisition_events # print the event log

## Procurement events
The procurement events includes all the events from purchase order, good receipts, and invoices.

We could leave it as is, but instead, we want to complement the events with the information that is stored in invoice.csv'

In [ ]:
procurements = pd.read_csv('procurement.csv',low_memory=False)
procurements

### Invoices.csv

In [ ]:
invoices = pd.read_csv('invoice.csv')
invoices

## Merging invoice.csv information into procurement.csv
We are going to add the invoice information to each event that relates to an invoice. To do that, we are going to use the key `Invoice_ID` to merge the invoice info to the invoice event, when the `Invoice_ID` columns match.

This would be a typical joint in a database. With Pandas, we use the `dataframe.merge()` function where we set the key, and the how="left" which means that we keep the procurement_events dataframe that we enrich with the invoice information.

In [ ]:
procurement_events = procurements.merge(invoices, on="Invoice_ID", how="left")
procurement_events.convert_dtypes() # Convert columns to best possible dtypes using dtypes supporting
procurement_events.to_csv('procurement_events.csv', index=None)
procurement_events

## Final Step: Create the P2P event log
Finally we append the requisition and the procurement event logs to create the final event log. Again, we can remove the events with a null `datetime`

In [ ]:
P2P_events = requisition_events.copy()
P2P_events = P2P_events.append(procurement_events)
P2P_events = P2P_events[P2P_events["datetime"].notna()] # removing rows with no datetime if any
P2P_events = P2P_events.convert_dtypes() # applying the best known types
P2P_events.to_csv('P2P_events.csv', index=None)
P2P_events.columns

In [ ]:
P2P_events